In [1]:
from pdb import set_trace
import os
import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
import warnings

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from constants import MODEL_PATHs
warnings.filterwarnings("ignore", message=".*To copy construct from a tensor.*")

/home/y-guo/miniconda3/envs/flexattention/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
nlp = None
num_parts = 8

In [3]:
adataset = "myriadlama"

In [4]:
from dataset import MyriadLamaDataset
amodel = "llama3.2_3b_it"
dataset = MyriadLamaDataset(model_name=amodel)

Dataset already exists at /net/tokyo100-10g/data/str01_01/y-guo/datasets/myriadlama/paraphrases_dataset. Loading from disk.


In [5]:
model_path = MODEL_PATHs.get(amodel, amodel)

In [6]:
dumpfile = f"{dataset.dataset_root}/per_prompt.feather"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [17]:
dataloader = dataset.get_dataloader(batch_size=8, shuffle=False)

In [18]:
dataloader

In [22]:
df = pd.DataFrame(columns=["uuid", "is_manual", "tpl_url","org_tpl_uri","answers", "question", "prompt", "prediction", "generation"])

In [88]:
few_shot_context = dataset.get_few_shot_examples()

In [20]:
i = iter(dataloader)

In [21]:
uuids, answers, all_paraphrases = next(i)

In [13]:
print(len(dataset.ds[0]["manual_paraphrases"]))

5


In [32]:
preds_in_batch = []
prompts_in_batch = []
is_manual_in_batch = []
tpl_url_in_batch = []
org_tpl_uri_in_batch = []
paraphrases_in_batch = []
generations_in_batch = []
predictions_in_batch = []

In [83]:
original_questions = all_paraphrases[0]

In [22]:
len(all_paraphrases[0])


8

In [37]:
prompts = dataset.construct_prompts(few_shot_context, original_questions)

In [38]:
prompts

['Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.\n\nQ: [MASK] is the higher-level concept of adit.\nA: tunnel\n\nQ: Marty Ehrlich became known as a [MASK] player.\nA: saxophone\n\nQ: Officially, the people living in Basel use the language [MASK] for communication.\nA: German\n\nQ: [MASK] consists of Mount Makiling.\nA: Luzon\n\nQ: Iowa and [MASK] are neighboring countries.\nA: Wisconsin\n\nQ: [MASK] is the country of origin of Peer Gynt.\nA:',
 'Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.\n\nQ: [MASK] is the higher-level concept of adit.\nA: tunnel\n\nQ: Marty Ehrlich became known as a [MASK] player.\nA: saxophone\n\nQ: Officially, the people living in Basel use the language [MASK] for communication.\nA: German\n\nQ: [MASK] consists of Mount Makiling.\nA: Luzon\n\nQ: Iowa and [MASK] are neighboring countries.\nA: Wisconsin\n\nQ: Rwanda and [MASK] have diplomatic friendly

In [50]:
all_paraphrases

[['Cardinal-Infante Ferdinand of Austria served as [MASK].',
  'Where was Gaspard Bauhin born? [MASK].',
  'John Feckenham served as [MASK].',
  'George Duke became known as a [MASK] player.',
  'Officially, the people living in Storuman Municipality use the language [MASK] for communication.',
  'Who does Peter Alliss work for? [MASK].',
  '[MASK] is the country in which Panagarh is located.',
  'Edward Burnett Tylor has work experience in the field of [MASK].'],
 ['Cardinal-Infante Ferdinand of Austria holds the political title of [MASK].',
  'Gaspard Bauhin was born in [MASK].',
  'John Feckenham holds the political title of [MASK].',
  'George Duke is a [MASK] player.',
  '[MASK] recognizes Storuman Municipality as its official language.',
  'Peter Alliss develops a career in [MASK].',
  'Which country is Panagarh located? [MASK].',
  'Edward Burnett Tylor works in the field of [MASK].'],
 ['Cardinal-Infante Ferdinand of Austria was sworn in as [MASK].',
  'Gaspard Bauhin saw the l

In [49]:
all_paraphrases = [list(paras) for paras in all_paraphrases]

In [51]:
all_prompts = []

In [64]:
paraphrases = next(iter(all_paraphrases))

In [53]:
paraphreases

['Cardinal-Infante Ferdinand of Austria served as [MASK].',
 'Where was Gaspard Bauhin born? [MASK].',
 'John Feckenham served as [MASK].',
 'George Duke became known as a [MASK] player.',
 'Officially, the people living in Storuman Municipality use the language [MASK] for communication.',
 'Who does Peter Alliss work for? [MASK].',
 '[MASK] is the country in which Panagarh is located.',
 'Edward Burnett Tylor has work experience in the field of [MASK].']

In [65]:
prompts = dataset.construct_prompts(few_shot_context, paraphrases)

In [67]:
all_prompts

[['Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.\n\nQ: [MASK] is the higher-level concept of adit.\nA: tunnel\n\nQ: Marty Ehrlich became known as a [MASK] player.\nA: saxophone\n\nQ: Officially, the people living in Basel use the language [MASK] for communication.\nA: German\n\nQ: [MASK] consists of Mount Makiling.\nA: Luzon\n\nQ: Iowa and [MASK] are neighboring countries.\nA: Wisconsin\n\nQ: Cardinal-Infante Ferdinand of Austria served as [MASK].\nA:',
  'Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.\n\nQ: [MASK] is the higher-level concept of adit.\nA: tunnel\n\nQ: Marty Ehrlich became known as a [MASK] player.\nA: saxophone\n\nQ: Officially, the people living in Basel use the language [MASK] for communication.\nA: German\n\nQ: [MASK] consists of Mount Makiling.\nA: Luzon\n\nQ: Iowa and [MASK] are neighboring countries.\nA: Wisconsin\n\nQ: Where was Gaspard Bauhin born?

In [66]:
all_prompts.append(prompts)

In [61]:
@torch.no_grad()
def ensemble_generation(prompt_sets, integration_method="max", weights=None):
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.generation_config.temperature = None
    model.generation_config.top_p = None
    model.generation_config.pad_token_id = tokenizer.eos_token_id

    generated = None
    max_new_tokens = 20

    all_inputs = []
    for prompts in prompt_sets:
        inputs = tokenizer(
            prompts, return_tensors="pt", 
            padding=True, truncation=True, 
            padding_side='left', return_attention_mask=True).to(model.device)
        all_inputs.append(inputs)

    all_input_ids = [inputs["input_ids"] for inputs in all_inputs]
    all_attention_masks = [inputs["attention_mask"] for inputs in all_inputs]
    for _ in range(max_new_tokens):
        logits_set = []
        with torch.no_grad():
            for idx, prompts in enumerate(prompt_sets):
                input_ids = all_input_ids[idx]
                attention_mask = all_attention_masks[idx]
                logits = model(input_ids, attention_mask=attention_mask).logits[:, -1, :]
                logits_set.append(logits)
            logits_set = torch.stack(logits_set)

        if integration_method == "avg":
            avg_logits = logits_set.mean(dim=0)
            next_token = torch.argmax(avg_logits, dim=-1).unsqueeze(1)
        elif integration_method == "max":    
            max_probs = logits_set.softmax(dim=-1).max(dim=0).values
            next_token = torch.argmax(max_probs, dim=-1).unsqueeze(1)
        elif integration_method == "weighted_avg":
            if weights is None:
                raise ValueError("Weights must be provided for weighted_avg integration.")
            weights = torch.tensor(weights).clone().detach().requires_grad_(False).to(logits_set.device)
            weights = weights / weights.sum(dim=0).unsqueeze(0)
            weighted_logits = (logits_set * weights.unsqueeze(-1)).sum(dim=0)
            next_token = torch.argmax(weighted_logits, dim=-1).unsqueeze(1)
        elif integration_method == "weighted_max":
            if weights is None:
                raise ValueError("Weights must be provided for weighted_max integration.")
            weights = torch.tensor(weights).clone().detach().requires_grad_(False).to(logits_set.device)
            argmax = weights.argmax(dim=0)
            max_logits = logits_set[argmax, torch.arange(logits_set.shape[1])]
            next_token = max_logits.argmax(dim=-1).unsqueeze(1)
        else:
            raise ValueError(f"Unknown integration method: {integration_method}")
        
        # Take the element-wise min across the two distributions
        # Append next token to input_ids for next round
        for i in range(len(all_input_ids)):            
            all_input_ids[i] = torch.cat([all_input_ids[i], next_token], dim=1)
            all_attention_masks[i] = torch.cat([all_attention_masks[i], torch.ones_like(next_token)], dim=1)

        if generated is None:
            generated = next_token
        else:
            generated = torch.cat([generated, next_token], dim=1)
        generated_texts = tokenizer.batch_decode(generated, skip_special_tokens=True)
        new_generated_texts = [gen.strip() for gen in generated_texts]
    return new_generated_texts


In [68]:
generations = ensemble_generation(all_prompts, integration_method="avg", weights=None)

In [69]:
generations

['Grand\n\nQ: The [MASK] of the city is the city hall.\nA: Town',
 'Switzerland\n\nQ: The [MASK] of the Roman Empire was the Emperor.\nA: Senate',
 'Bishop\n\nQ: The [MASK] of the city is the city hall.\nA: building',
 'keyboard\n\nQ: The [MASK] of the city is the city hall.\nA: town',
 'Sami\n\nQ: [MASK] is the name of the largest island in the Mediterranean Sea.',
 'The Open\n\nQ: The [MASK] of the city is the city hall.\nA: municipal',
 'India\n\nQ: [MASK] is the name of the largest island in the Mediterranean Sea.\nA',
 'anthropology\n\nQ: The [MASK] of the city is a major tourist attraction.\nA: landmark']

In [70]:
predictions = [gen.strip().split('\n')[0] for gen in generations]

In [71]:
items = {
                "uuid": uuids,
                "paraphrases": list(zip(*all_paraphrases)),
                "prompts": list(zip(*all_prompts)),
                "answers": answers,
                "prediction": predictions,
                "generation": generations,
            }

In [74]:
predictions

['Grand',
 'Switzerland',
 'Bishop',
 'keyboard',
 'Sami',
 'The Open',
 'India',
 'anthropology']

In [72]:
items

{'uuid': ['782f4ee9a1f6d31-a0fd1db0ac93f0f',
  'e772dad2b624394-50894113c6516c9',
  '782f4ee9a1f6d31-d2aa69bf1c0cb15',
  '04dc00943c6cc89-e962371af6d24da',
  '2f6cd7e4e9d7572-ba71ebac2d1c72a',
  '9f1ea9fa44e6c0b-91fd2e466697790',
  'dba688a59d61da0-d706af18a32f6c4',
  'fd915def246a0be-29c52ddc2479096'],
 'paraphrases': [('Cardinal-Infante Ferdinand of Austria served as [MASK].',
   'Cardinal-Infante Ferdinand of Austria holds the political title of [MASK].',
   'Cardinal-Infante Ferdinand of Austria was sworn in as [MASK].',
   'What position does Cardinal-Infante Ferdinand of Austria hold? [MASK].',
   'Cardinal-Infante Ferdinand of Austria has the position of [MASK].',
   'What position does Cardinal-Infante Ferdinand of Austria hold at work? [MASK].',
   'Cardinal-Infante Ferdinand of Austria pledged their commitment to the role of [MASK].',
   'Cardinal-Infante Ferdinand of Austria took up the mantle of [MASK].',
   'Cardinal-Infante Ferdinand of Austria functioned as [MASK].',
   